## Benchmark model

In this notebook a Simple Moving Average model will be created in order to have a benchmark for my DeepAR based model (to be tuned in the next notebook).

### Hyperparameters

DeepAR is the model of choice of this project.
This model expects input data to be already test-train splitted.
A big part of the model design has to be done looking close at data.
More specifically, defining these two hyperparameters about the data:
* Context length
* Prediction length

### Prediction length

This is the length of the time series future predictions in days. It will be conveniently set to 5 days (exactly a week of trading hours) because a shorter interval would be of little significance.
A longer interval could be interesting from an application point of view, but it can be challenging in terms of model performances.

### Context length

Context length can be either:
* designed on patterns or seasonality observed in the data, if any is present;
* chosen as a fixed value. This will be my choice, and it will be the same as the moving average window, in order to have a good reference metrics, applicable to both this model and the benchmark model.

To explore this second option, we will refer to what we've found during the EDA stage.

In [97]:
prediction_length = [10, 20, 50]
context_length = [10, 20, 50]

### Train, test and validation split

Time series will be all trimmed from `data_start` in `1.ExploratoryDataAnalysis.ipy`, according to DeepAR documentation, train time series should be set as the size of the entire time series less the prediction length.
Validation length will exactly equal to prediction length.

In [98]:
from data.data_utils import train_test_valid_split

#### IBM Stock train, test and validation split

In [99]:
df_ibm = df_ibm.loc[data_start:].copy()

In [100]:
df_ibm_train, df_ibm_test, df_ibm_valid = train_test_valid_split(df_ibm, prediction_length=prediction_length[1])
print(len(df_ibm), len(df_ibm_train), len(df_ibm_test), len(df_ibm_valid))

4170 4130 4150 20


In [101]:
df_ibm_train.head()

,Open,High,Low,Close,Adj Close,Volume,10_ac_ma,20_ac_ma,50_ac_ma,10_ac_bb_u,10_ac_bb_l,20_ac_bb_u,20_ac_bb_l,50_ac_bb_u,50_ac_bb_l
Date,,,,,,,,,,,,,,,
2004-08-19,84.750000,85.349998,84.449997,84.889999,54.979382,4704500,54.397126,54.958798,55.913001,55.525647,53.268605,56.500281,53.417314,59.046566,52.779437
2004-08-20,84.519997,85.250000,84.519997,85.250000,55.212502,4501400,54.511759,54.977556,55.852774,55.720874,53.302644,56.521944,53.433168,58.920109,52.785438
2004-08-23,85.230003,85.449997,84.650002,84.650002,54.823910,4260600,54.582999,54.969128,55.779992,55.771068,53.394930,56.515013,53.423244,58.766151,52.793832
2004-08-24,85.000000,85.150002,84.349998,84.709999,54.862782,2710400,54.564866,54.937116,55.713028,55.726906,53.402826,56.462835,53.411398,58.625949,52.800108
2004-08-25,85.000000,85.269997,84.550003,85.070000,55.095955,4405600,54.654243,54.917732,55.644653,55.829713,53.478772,56.423940,53.411524,58.447571,52.841736


In [102]:
df_ibm_test.head()

,Open,High,Low,Close,Adj Close,Volume,10_ac_ma,20_ac_ma,50_ac_ma,10_ac_bb_u,10_ac_bb_l,20_ac_bb_u,20_ac_bb_l,50_ac_bb_u,50_ac_bb_l
Date,,,,,,,,,,,,,,,
2004-08-19,84.750000,85.349998,84.449997,84.889999,54.979382,4704500,54.397126,54.958798,55.913001,55.525647,53.268605,56.500281,53.417314,59.046566,52.779437
2004-08-20,84.519997,85.250000,84.519997,85.250000,55.212502,4501400,54.511759,54.977556,55.852774,55.720874,53.302644,56.521944,53.433168,58.920109,52.785438
2004-08-23,85.230003,85.449997,84.650002,84.650002,54.823910,4260600,54.582999,54.969128,55.779992,55.771068,53.394930,56.515013,53.423244,58.766151,52.793832
2004-08-24,85.000000,85.150002,84.349998,84.709999,54.862782,2710400,54.564866,54.937116,55.713028,55.726906,53.402826,56.462835,53.411398,58.625949,52.800108
2004-08-25,85.000000,85.269997,84.550003,85.070000,55.095955,4405600,54.654243,54.917732,55.644653,55.829713,53.478772,56.423940,53.411524,58.447571,52.841736


In [103]:
df_ibm_valid.head()

,Open,High,Low,Close,Adj Close,Volume,10_ac_ma,20_ac_ma,50_ac_ma,10_ac_bb_u,10_ac_bb_l,20_ac_bb_u,20_ac_bb_l,50_ac_bb_u,50_ac_bb_l
Date,,,,,,,,,,,,,,,
2021-02-12,121.000000,121.360001,120.089996,120.800003,120.800003,3868900,120.200284,121.306254,122.992767,123.628698,116.771870,129.077036,113.535473,129.279414,116.706120
2021-02-16,120.150002,120.599998,119.360001,120.070000,120.070000,6639800,120.312235,120.974906,122.934634,123.630615,116.993855,128.331988,113.617823,129.275412,116.593856
2021-02-17,119.269997,120.559998,119.019997,119.970001,119.970001,3959100,120.522736,120.607473,122.894434,123.387170,117.658302,127.337892,113.877054,129.285204,116.503663
2021-02-18,120.500000,120.940002,119.699997,120.730003,120.730003,5398100,120.840816,120.225739,122.798580,122.801605,118.880026,125.884348,114.567129,129.172198,116.424963
2021-02-19,120.750000,120.760002,118.379997,118.989998,118.989998,6578700,120.797401,119.679540,122.717268,122.910881,118.683921,123.044776,116.314304,129.180605,116.253931


#### Apple Inc. Stock train test split

In [104]:
df_aapl = df_aapl.loc[data_start:].copy()

In [105]:
df_aapl_train, df_aapl_test, df_aapl_valid = train_test_valid_split(df_aapl, prediction_length=prediction_length[1])
print(len(df_aapl), len(df_aapl_train), len(df_aapl_test), len(df_aapl_valid))

4170 4130 4150 20


In [106]:
df_aapl_train.head()

,Open,High,Low,Close,Adj Close,Volume,10_ac_ma,20_ac_ma,50_ac_ma,10_ac_bb_u,10_ac_bb_l,20_ac_bb_u,20_ac_bb_l,50_ac_bb_u,50_ac_bb_l
Date,,,,,,,,,,,,,,,
2004-08-19,0.562679,0.568929,0.542143,0.548393,0.472366,388920000,0.473627,0.481141,0.484422,0.491197,0.456057,0.504857,0.457425,0.519835,0.449008
2004-08-20,0.548393,0.553393,0.544464,0.550000,0.473750,316780800,0.475196,0.481218,0.484606,0.488983,0.461409,0.504821,0.457614,0.519691,0.449521
2004-08-23,0.551071,0.558393,0.546429,0.555000,0.478057,254660000,0.476395,0.481079,0.484711,0.488652,0.464139,0.504725,0.457433,0.519683,0.449738
2004-08-24,0.558214,0.570536,0.556964,0.570536,0.491439,374136000,0.477057,0.480710,0.485274,0.491797,0.462317,0.503401,0.458019,0.519742,0.450806
2004-08-25,0.569107,0.591964,0.566607,0.590179,0.508359,505618400,0.480195,0.481310,0.486000,0.504872,0.455517,0.506265,0.456355,0.520858,0.451141


In [107]:
df_aapl_test.head()

,Open,High,Low,Close,Adj Close,Volume,10_ac_ma,20_ac_ma,50_ac_ma,10_ac_bb_u,10_ac_bb_l,20_ac_bb_u,20_ac_bb_l,50_ac_bb_u,50_ac_bb_l
Date,,,,,,,,,,,,,,,
2004-08-19,0.562679,0.568929,0.542143,0.548393,0.472366,388920000,0.473627,0.481141,0.484422,0.491197,0.456057,0.504857,0.457425,0.519835,0.449008
2004-08-20,0.548393,0.553393,0.544464,0.550000,0.473750,316780800,0.475196,0.481218,0.484606,0.488983,0.461409,0.504821,0.457614,0.519691,0.449521
2004-08-23,0.551071,0.558393,0.546429,0.555000,0.478057,254660000,0.476395,0.481079,0.484711,0.488652,0.464139,0.504725,0.457433,0.519683,0.449738
2004-08-24,0.558214,0.570536,0.556964,0.570536,0.491439,374136000,0.477057,0.480710,0.485274,0.491797,0.462317,0.503401,0.458019,0.519742,0.450806
2004-08-25,0.569107,0.591964,0.566607,0.590179,0.508359,505618400,0.480195,0.481310,0.486000,0.504872,0.455517,0.506265,0.456355,0.520858,0.451141


In [108]:
df_aapl_valid.head()

,Open,High,Low,Close,Adj Close,Volume,10_ac_ma,20_ac_ma,50_ac_ma,10_ac_bb_u,10_ac_bb_l,20_ac_bb_u,20_ac_bb_l,50_ac_bb_u,50_ac_bb_l
Date,,,,,,,,,,,,,,,
2021-02-12,134.350006,135.529999,133.690002,135.369995,135.369995,60029300,135.522359,135.666208,131.056535,137.914301,133.130416,144.087912,127.244505,142.327544,119.785526
2021-02-16,135.490005,136.009995,132.789993,133.190002,133.190002,80576300,135.447374,135.978194,131.262408,138.093331,132.801417,143.449006,128.507381,142.298640,120.226176
2021-02-17,131.250000,132.220001,129.470001,130.839996,130.839996,97918500,135.052515,136.138230,131.424077,138.995849,131.109180,142.966393,129.310068,142.185129,120.663025
2021-02-18,129.199997,130.000000,127.410004,129.710007,129.710007,96856700,134.649500,136.032081,131.576925,139.821352,129.477647,143.199620,128.864541,142.007567,121.146283
2021-02-19,130.240005,130.710007,128.800003,129.869995,129.869995,87668800,133.917999,135.692292,131.703018,139.545200,128.290798,143.360206,128.024378,141.887890,121.518145


#### Amazon Stock train test split

In [109]:
df_amzn = df_amzn.loc[data_start:].copy()

In [110]:
df_amzn_train, df_amzn_test, df_amzn_valid = train_test_valid_split(df_amzn, prediction_length=prediction_length[1])
print(len(df_amzn), len(df_amzn_train), len(df_amzn_test), len(df_amzn_valid))

4170 4130 4150 20


In [111]:
df_amzn_train.head()

,Open,High,Low,Close,Adj Close,Volume,10_ac_ma,20_ac_ma,50_ac_ma,10_ac_bb_u,10_ac_bb_l,20_ac_bb_u,20_ac_bb_l,50_ac_bb_u,50_ac_bb_l
Date,,,,,,,,,,,,,,,
2004-08-19,40.259998,40.480000,36.349998,38.630001,38.630001,12696100,37.106001,37.6520,44.9828,39.862539,34.349462,40.417963,34.886038,57.693587,32.272014
2004-08-20,38.459999,39.910000,38.110001,39.509998,39.509998,6790800,37.508000,37.6285,44.7682,40.386916,34.629084,40.318115,34.938885,57.479021,32.057379
2004-08-23,39.889999,40.000000,39.110001,39.450001,39.450001,5532600,37.921000,37.6620,44.5584,40.581550,35.260450,40.427620,34.896380,57.267085,31.849715
2004-08-24,39.720001,39.930000,38.320000,39.049999,39.049999,7640400,38.116000,37.6590,44.3544,40.794891,35.437110,40.418129,34.899871,57.083144,31.625656
2004-08-25,39.060001,40.490002,38.160000,40.299999,40.299999,7254800,38.490000,37.7755,44.1582,41.246567,35.733433,40.776113,34.774887,56.827117,31.489283


In [112]:
df_amzn_test.head()

,Open,High,Low,Close,Adj Close,Volume,10_ac_ma,20_ac_ma,50_ac_ma,10_ac_bb_u,10_ac_bb_l,20_ac_bb_u,20_ac_bb_l,50_ac_bb_u,50_ac_bb_l
Date,,,,,,,,,,,,,,,
2004-08-19,40.259998,40.480000,36.349998,38.630001,38.630001,12696100,37.106001,37.6520,44.9828,39.862539,34.349462,40.417963,34.886038,57.693587,32.272014
2004-08-20,38.459999,39.910000,38.110001,39.509998,39.509998,6790800,37.508000,37.6285,44.7682,40.386916,34.629084,40.318115,34.938885,57.479021,32.057379
2004-08-23,39.889999,40.000000,39.110001,39.450001,39.450001,5532600,37.921000,37.6620,44.5584,40.581550,35.260450,40.427620,34.896380,57.267085,31.849715
2004-08-24,39.720001,39.930000,38.320000,39.049999,39.049999,7640400,38.116000,37.6590,44.3544,40.794891,35.437110,40.418129,34.899871,57.083144,31.625656
2004-08-25,39.060001,40.490002,38.160000,40.299999,40.299999,7254800,38.490000,37.7755,44.1582,41.246567,35.733433,40.776113,34.774887,56.827117,31.489283


In [113]:
df_amzn_valid.head()

,Open,High,Low,Close,Adj Close,Volume,10_ac_ma,20_ac_ma,50_ac_ma,10_ac_bb_u,10_ac_bb_l,20_ac_bb_u,20_ac_bb_l,50_ac_bb_u,50_ac_bb_l
Date,,,,,,,,,,,,,,,
2021-02-12,3250.000000,3280.250000,3233.310059,3277.709961,3277.709961,2329300,3317.291968,3277.852979,3222.085181,3389.552311,3245.031625,3419.109022,3136.596935,3375.172588,3068.997774
2021-02-16,3254.050049,3308.300049,3253.590088,3268.949951,3268.949951,2574700,3309.898975,3286.087976,3223.393579,3385.571219,3234.226730,3401.585069,3170.590883,3376.951228,3069.835930
2021-02-17,3263.600098,3320.909912,3259.500000,3308.639893,3308.639893,3297500,3302.762964,3295.481970,3225.831777,3360.352910,3245.173017,3381.043039,3209.920902,3380.877493,3070.786062
2021-02-18,3282.419922,3338.000000,3273.939941,3328.229980,3328.229980,3027400,3304.332959,3298.724475,3229.144775,3363.927436,3244.738482,3384.078129,3213.370821,3385.745319,3072.544232
2021-02-19,3328.229980,3333.500000,3245.750000,3249.899902,3249.899902,4305200,3296.222949,3295.869971,3230.982773,3361.491566,3230.954332,3383.838238,3207.901704,3386.327284,3075.638263


#### Alphabet Inc. Stock train test split

In [114]:
df_googl = df_googl.loc[data_start:].copy()

In [115]:
df_googl_train, df_googl_test, df_googl_valid = train_test_valid_split(df_googl, prediction_length=prediction_length[1])
print(len(df_googl), len(df_googl_train), len(df_googl_test), len(df_googl_valid))

4170 4130 4150 20


In [116]:
df_googl_train.head()

,Open,High,Low,Close,Adj Close,Volume,10_ac_ma,20_ac_ma,50_ac_ma,10_ac_bb_u,10_ac_bb_l,20_ac_bb_u,20_ac_bb_l,50_ac_bb_u,50_ac_bb_l
Date,,,,,,,,,,,,,,,
2004-08-19,50.050049,52.082081,48.028027,50.220219,50.220219,44659096,52.432933,52.692943,63.586987,55.781053,49.084813,56.933168,48.452718,88.331593,38.842381
2004-08-20,50.555557,54.594597,50.300301,54.209209,54.209209,22834343,52.432933,52.692943,63.586987,55.781053,49.084813,56.933168,48.452718,88.331593,38.842381
2004-08-23,55.430431,56.796799,54.579578,54.754753,54.754753,18256126,52.432933,52.692943,63.586987,55.781053,49.084813,56.933168,48.452718,88.331593,38.842381
2004-08-24,55.675674,55.855858,51.836838,52.487488,52.487488,15247337,52.432933,52.692943,63.586987,55.781053,49.084813,56.933168,48.452718,88.331593,38.842381
2004-08-25,52.532532,54.054054,51.991993,53.053055,53.053055,9188602,52.432933,52.692943,63.586987,55.781053,49.084813,56.933168,48.452718,88.331593,38.842381


In [117]:
df_googl_test.head()

,Open,High,Low,Close,Adj Close,Volume,10_ac_ma,20_ac_ma,50_ac_ma,10_ac_bb_u,10_ac_bb_l,20_ac_bb_u,20_ac_bb_l,50_ac_bb_u,50_ac_bb_l
Date,,,,,,,,,,,,,,,
2004-08-19,50.050049,52.082081,48.028027,50.220219,50.220219,44659096,52.432933,52.692943,63.586987,55.781053,49.084813,56.933168,48.452718,88.331593,38.842381
2004-08-20,50.555557,54.594597,50.300301,54.209209,54.209209,22834343,52.432933,52.692943,63.586987,55.781053,49.084813,56.933168,48.452718,88.331593,38.842381
2004-08-23,55.430431,56.796799,54.579578,54.754753,54.754753,18256126,52.432933,52.692943,63.586987,55.781053,49.084813,56.933168,48.452718,88.331593,38.842381
2004-08-24,55.675674,55.855858,51.836838,52.487488,52.487488,15247337,52.432933,52.692943,63.586987,55.781053,49.084813,56.933168,48.452718,88.331593,38.842381
2004-08-25,52.532532,54.054054,51.991993,53.053055,53.053055,9188602,52.432933,52.692943,63.586987,55.781053,49.084813,56.933168,48.452718,88.331593,38.842381


In [118]:
df_googl_valid.head()

,Open,High,Low,Close,Adj Close,Volume,10_ac_ma,20_ac_ma,50_ac_ma,10_ac_bb_u,10_ac_bb_l,20_ac_bb_u,20_ac_bb_l,50_ac_bb_u,50_ac_bb_l
Date,,,,,,,,,,,,,,,
2021-02-12,2079.770020,2099.500000,2073.570068,2095.030029,2095.030029,948500,2044.369971,1945.714978,1834.833992,2193.046774,1895.693167,2186.006374,1705.423582,2076.437444,1593.230539
2021-02-16,2091.750000,2145.139893,2091.750000,2110.699951,2110.699951,1550200,2066.132971,1964.868976,1840.548591,2174.673734,1957.592208,2192.710977,1737.026974,2094.405644,1586.691538
2021-02-17,2093.979980,2122.439941,2088.350098,2118.620117,2118.620117,1014700,2086.082983,1981.576483,1846.484194,2126.467383,2045.698584,2202.624080,1760.528886,2112.159135,1580.809254
2021-02-18,2105.000000,2121.989990,2093.330078,2105.810059,2105.810059,1127300,2090.776001,1992.863489,1852.125195,2127.885024,2053.666978,2215.137461,1770.589516,2127.626424,1576.623967
2021-02-19,2107.790039,2118.580078,2083.429932,2088.810059,2088.810059,1896900,2094.294019,2003.096490,1857.560796,2120.949812,2067.638226,2223.129983,1783.062998,2140.850093,1574.271498


### Metrics computation

Computing metrics on the benchmark model will give me a good reference to evaluate the deep learning model after training.

As can be presumed and seen from data prints, first `n` values of validation data are `NaN` so, it will be better just to exclude the first `n-1` values from the validation set to avoid misleading values to be catch in metrics evaluation. 

In [119]:
from sklearn.metrics import mean_absolute_error, mean_squared_error, mean_absolute_percentage_error, r2_score

#### IBM stock

In [120]:
#n = 10
#ma_str = str(n)+'_ac_ma'

In [121]:
#ibm_ma_mse_loss = mean_squared_error(df_ibm_valid.iloc[n-1:]['Adj Close'], df_ibm_valid.iloc[n-1:][ma_str])

In [122]:
#print(ibm_ma_mse_loss)

In [123]:
n = 20
ma_str = str(n)+'_ac_ma'

Mean Absolute Error

In [124]:
ibm_ma_mae_loss = mean_absolute_error(df_ibm_valid.iloc[:]['Adj Close'], df_ibm_valid[:][ma_str])

In [125]:
print(ibm_ma_mae_loss)

1.967003059387207


Root Mean Squared Error

In [126]:
ibm_ma_mse_loss = mean_squared_error(df_ibm_valid.iloc[:]['Adj Close'], df_ibm_valid[:][ma_str], squared=False)

In [127]:
print(ibm_ma_mse_loss)

2.6612423160137473


Mean Absolute Percentage Error

In [128]:
ibm_ma_map_loss = mean_absolute_percentage_error(df_ibm_valid.iloc[:]['Adj Close'], df_ibm_valid[:][ma_str])

In [129]:
print(ibm_ma_map_loss)

0.015796526669715504


R<sup>2</sup> score

In [130]:
ibm_ma_r2_score = r2_score(df_ibm_valid.iloc[:]['Adj Close'], df_ibm_valid[:][ma_str])

In [131]:
print(ibm_ma_r2_score)

0.034349484114639006


In [132]:
#n = 50
#ma_str = str(n)+'_ac_ma'

In [133]:
#ibm_ma_mse_loss = mean_squared_error(df_ibm_valid.iloc[n-1:]['Adj Close'], df_ibm_valid[n-1:][ma_str])

In [134]:
#print(ibm_ma_mse_loss)

#### Apple Inc. stock

In [135]:
#n = 10
#ma_str = str(n)+'_ac_ma'

In [136]:
#aapl_ma_mse_loss = mean_squared_error(df_aapl_valid.iloc[n-1:]['Adj Close'], df_aapl_valid.iloc[n-1:][ma_str])

In [137]:
#print(aapl_ma_mse_loss)

In [138]:
n = 20
ma_str = str(n)+'_ac_ma'

Mean Absolute Error

In [139]:
aapl_ma_mae_loss = mean_absolute_error(df_aapl_valid.iloc[:]['Adj Close'], df_aapl_valid[:][ma_str])

In [140]:
print(aapl_ma_mae_loss)

6.613441658020018


##### Root Mean Squared Error

In [141]:
aapl_ma_rmse_loss = mean_squared_error(df_aapl_valid.iloc[:]['Adj Close'], df_aapl_valid[:][ma_str], squared=False)

In [142]:
print(aapl_ma_rmse_loss)

7.211482715323648


##### Mean Absolute Percentage Error

In [143]:
aapl_ma_map_loss = mean_absolute_percentage_error(df_aapl_valid.iloc[:]['Adj Close'], df_aapl_valid[:][ma_str])

In [144]:
print(aapl_ma_map_loss)

0.0536576634924276


R<sup>2</sup> score

In [145]:
aapl_ma_r2_score = r2_score(df_aapl_valid.iloc[:]['Adj Close'], df_aapl_valid[:][ma_str])

In [146]:
print(aapl_ma_r2_score)

-1.1732421328209632


In [147]:
#n = 50
#ma_str = str(n)+'_ac_ma'

In [148]:
#aapl_ma_mse_loss = mean_squared_error(df_aapl_valid.iloc[n-1:]['Adj Close'], df_aapl_valid[n-1:][ma_str])

In [149]:
#print(aapl_ma_mse_loss)

#### Amazon.com stock

In [150]:
#n = 10
#ma_str = str(n)+'_ac_ma'

In [151]:
#amzn_ma_mse_loss = mean_squared_error(df_amzn_valid.iloc[n-1:]['Adj Close'], df_amzn_valid.iloc[n-1:][ma_str])

In [152]:
#print(amzn_ma_mse_loss)

In [153]:
n = 20
ma_str = str(n)+'_ac_ma'

Mean Absolute Error

In [154]:
amzn_ma_mae_loss = mean_absolute_error(df_amzn_valid.iloc[:]['Adj Close'], df_amzn_valid[:][ma_str])

In [155]:
print(amzn_ma_mae_loss)

109.5103369140625


Root Mean Squared Error

In [156]:
amzn_ma_rmse_loss = mean_squared_error(df_amzn_valid.iloc[:]['Adj Close'], df_amzn_valid[:][ma_str], squared=False)

In [157]:
print(amzn_ma_rmse_loss)

133.2964148560102


##### Mean Absolute Percentage Error

In [158]:
amzn_ma_map_loss = mean_absolute_percentage_error(df_amzn_valid.iloc[:]['Adj Close'], df_amzn_valid[:][ma_str])

In [159]:
print(amzn_ma_map_loss)

0.03572878093904378


R<sup>2</sup> score

In [160]:
amzn_ma_r2_score = r2_score(df_amzn_valid.iloc[:]['Adj Close'], df_amzn_valid[:][ma_str])

In [161]:
print(amzn_ma_r2_score)

-0.47148656289280555


In [162]:
#n = 50
#ma_str = str(n)+'_ac_ma'

In [163]:
#amzn_ma_mse_loss = mean_squared_error(df_amzn_valid.iloc[n-1:]['Adj Close'], df_amzn_valid[n-1:][ma_str])

In [164]:
#print(amzn_ma_mse_loss)

#### Alphabet Inc. stock

In [165]:
#n = 10
#ma_str = str(n)+'_ac_ma'

In [166]:
#googl_ma_mse_loss = mean_squared_error(df_googl_valid.iloc[n-1:]['Adj Close'], df_googl_valid.iloc[n-1:][ma_str])

In [167]:
#print(googl_ma_mse_loss)

In [168]:
n = 20
ma_str = str(n)+'_ac_ma'

Mean Absolute Error

In [169]:
googl_ma_mae_loss = mean_absolute_error(df_googl_valid.iloc[n-1:]['Adj Close'], df_googl_valid[n-1:][ma_str])

In [170]:
print(googl_ma_mae_loss)

13.308020019531341


Root Mean Squared Error

In [171]:
googl_ma_rmse_loss = mean_squared_error(df_googl_valid.iloc[n-1:]['Adj Close'], df_googl_valid[n-1:][ma_str], squared=False)

In [172]:
print(googl_ma_rmse_loss)

13.308020019531341


##### Mean Absolute Percentage Error

In [173]:
googl_ma_map_loss = mean_absolute_percentage_error(df_googl_valid.iloc[:]['Adj Close'], df_googl_valid[:][ma_str])

In [174]:
print(googl_ma_map_loss)

0.026854600167504194


R<sup>2</sup> score

In [175]:
googl_ma_r2_score = r2_score(df_googl_valid.iloc[:]['Adj Close'], df_googl_valid[:][ma_str])

In [176]:
print(googl_ma_r2_score)

-3.286440773721985


In [177]:
#n = 50
#ma_str = str(n)+'_ac_ma'

In [178]:
#googl_ma_mse_loss = mean_squared_error(df_googl_valid.iloc[n-1:]['Adj Close'], df_googl_valid[n-1:][ma_str])

In [179]:
#print(googl_ma_mse_loss)

Volatility

In [180]:
print(volatility(df_ibm_valid['Adj Close'], n))

0.3860070735539271


In [181]:
print(volatility(df_aapl_valid['Adj Close'], n))

1.259468809490453


In [182]:
print(volatility(df_amzn_valid['Adj Close'], n))

635.5168004043696


In [183]:
print(volatility(df_googl_valid['Adj Close'], n))

62.93514491077347


As expected, loss augments as we observe moving average on larger windows.
Also, we can observe that Amazon.com and Alphabet Inc. have greater losses, that also corresponds to higher volatility.